In [ ]:
import tiktoken
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
import numpy as np
import bs4
from langchain_community.document_loaders import WebBaseLoader
from pprint import pprint
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

### トークン数取得

In [ ]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """文字列のトークン数を返す"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))

    return num_tokens

In [ ]:
# トークン数確認
print('質問のトークン数', num_tokens_from_string(question, 'cl100k_base'))

### 埋め込みの作成

In [ ]:
# 埋め込みモデルの用意
embd = OpenAIEmbeddings()

In [ ]:
query_result = embd.embed_query(question)
print('質問埋め込みベクトルの次元', len(query_result))
document_result = embd.embed_query(document)
print('文書埋め込みベクトルの次元', len(document_result))

### コサイン類似度の計算

In [ ]:
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec_1 = np.linalg.norm(vec1)
    norm_vec_2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec_1 * norm_vec_2)

In [ ]:
similarity = cosine_similarity(query_result, document_result)
print('質問と文書のコサイン類似度', similarity)

### Web文書の取得

In [ ]:
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    )
)
blog_docs = loader.load()

In [ ]:
print('文書数:', len(blog_docs))
pprint(blog_docs[0].page_content)

### 文書の分割

In [ ]:
# トークン数を基準とした分割
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, chunk_overlap=50
)

In [ ]:
# 分割及び、分割数確認
splits = text_splitter.split_documents(blog_docs)
print('分割数:', len(splits))

### インデックスの作成

In [ ]:
vectorstore = Chroma().from_documents(documents=splits, embedding=OpenAIEmbeddings())

### リトリーバの作成

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

### 文書検索

In [ ]:
docs = retriever.get_relevant_documents("What is Task Decomposition?")

In [ ]:
print('取得関連文書数', len(docs))
pprint(docs[0].page_content)

### プロンプトテンプレートの作成

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
print(prompt.messages[0].prompt.template)

### LLMの用意

In [ ]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

### チェーン(文書直渡し)の作成

In [ ]:
chain = prompt | llm

### 実行(文書直渡し)

In [ ]:
pprint(chain.invoke({"context": docs, "question": "What is Task Decomposition?"}).content)

### RAGチェーンの作成

In [ ]:
# RAG用プロンプト取得
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [ ]:
print(prompt_hub_rag.messages[0].prompt.template)

### RAGチェーン実行

In [ ]:
rag_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

In [ ]:
pprint(rag_chain.invoke("What is Task Decomposition?"))